In [ ]:
import glob
import os
import pickle
import xml.etree.ElementTree as ET
from os import listdir, getcwd
from os.path import join

In [ ]:
# this is where your export VOC dataset is
VOC_DIRS = ['/usr/src/app/yolov5/p1-PascalVOC-export']

# this is where your outputs will be written to
LABEL_DIR = '/usr/src/app/yolov5/p1-PascalVOC-export/labels/'
IMG_DIR = '/usr/src/app/yolov5/p1-PascalVOC-export/images/'

classes = ['TIE', 'MF']

In [ ]:
def getImagesInDir(dir_path):
    image_list = []
    for filename in glob.glob(dir_path + '/JPEGImages/*.jpg'):
        image_list.append(filename)

    return image_list

In [ ]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
def convert_annotation(dir_path, label_path, image_path):
    basename = os.path.basename(image_path)
    basename_no_ext = os.path.splitext(basename)[0]
    print("basebane_no_ext: ", basename_no_ext)

    in_file = open(dir_path + '/Annotations/' + basename_no_ext + '.xml')
    label_file = open(label_path + basename_no_ext + '.txt', 'w')
    tree = ET.parse(in_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        cls = obj.find('name').text
        if cls not in classes or int(difficult)==1:
            continue
        cls_id = classes.index(cls)
        xmlbox = obj.find('bndbox')
        b = (float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text))
        bb = convert((w,h), b)
        label_file.write(str(cls_id) + " " + " ".join([str(a) for a in bb]) + '\n')

In [ ]:
if not os.path.exists(LABEL_DIR):
    print("creating: ", LABEL_DIR)
    os.makedirs(LABEL_DIR)

In [ ]:
if not os.path.exists(IMG_DIR):
    print("creating: ", IMG_DIR)
    os.makedirs(IMG_DIR)

In [ ]:
import shutil
from shutil import copyfile


In [ ]:
for dir_path in VOC_DIRS:
    image_paths = getImagesInDir(dir_path)
    print("found ", len(image_paths), " images")
    for image_path in image_paths:
        convert_annotation(dir_path, LABEL_DIR, image_path)
        copyfile(image_path,IMG_DIR + os.path.basename(image_path))
        print(image_path)


    print("Finished processing: " + dir_path)